# Q3 - R section

## First Machine ##

In [45]:
library(limma)
library(edgeR)

In [46]:
normal_counts <- read.csv("train_normal_counts.csv")

In [47]:
head(normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.820135,6.5462994,6.6040504,6.480745,6.550016,6.5692529,5.9652146,6.436726,6.3378589,6.3677048,⋯,6.85952138,6.3703867,6.6352203,6.693334,6.26633823,6.6089237,6.34423435,6.296354,6.485184,5.965571
2,-1.060061,0.5821648,-0.8650363,-1.083676,-1.222374,0.7672549,-0.2056643,1.225274,0.3287041,-0.9022011,⋯,0.03594184,-0.5651604,-0.4658174,1.603149,0.08898988,-0.7573986,-0.02658513,-2.568864,-0.486609,1.612375
3,4.388400,3.7520898,4.3514891,4.361634,4.534941,4.1504701,3.1093774,4.104779,4.6099902,4.1480104,⋯,4.12340468,4.7392966,4.3778794,4.473400,4.31945643,4.2744503,4.17931917,4.381449,4.331904,4.133821
4,4.080172,4.6451746,4.0721368,4.313540,4.370763,4.1660389,4.8092861,4.411547,4.3253607,4.5060579,⋯,4.76204644,4.1208213,4.2298584,4.755317,4.38806704,4.0425768,4.43879576,4.365810,4.090491,4.111056
5,2.564430,3.8408991,3.1431376,3.120196,3.512952,3.7570108,3.9527651,3.099743,3.3085263,3.9820712,⋯,3.95559973,2.9326175,3.6972213,4.027257,3.97372734,2.1264084,3.81471713,2.965115,2.697188,4.150662
6,3.552685,3.2010747,4.0374758,1.941859,2.517867,3.2536535,2.7857226,3.357725,1.9136666,2.2021356,⋯,2.72896408,2.8246104,2.2057995,1.576431,2.97651515,3.4239312,2.27774891,3.408416,3.630074,2.975845


In [48]:
dim(normal_counts)

[1] 17396   134

In [49]:
meta_data <- read.csv("train_meta_data.csv")

In [50]:
head(meta_data)

,Simplified_class
,<chr>
1,Normal
2,Advanced_fibrosis
3,Normal
4,Normal
5,Non_advanced_Fibrosis
6,Normal


In [51]:
meta_data$Simplified_class[meta_data$Simplified_class != 'Normal'] <- 'Fibrosis'

In [52]:
labels <- factor(meta_data$Simplified_class)

In [53]:
print(labels)

  [1] Normal   Fibrosis Normal   Normal   Fibrosis Normal   Fibrosis Fibrosis
  [9] Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis
 [17] Normal   Normal   Normal   Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [25] Fibrosis Fibrosis Normal   Fibrosis Normal   Fibrosis Fibrosis Fibrosis
 [33] Fibrosis Normal   Normal   Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis
 [41] Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis
 [49] Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis Normal   Fibrosis
 [57] Fibrosis Fibrosis Normal   Normal   Normal   Fibrosis Fibrosis Fibrosis
 [65] Normal   Normal   Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [73] Normal   Fibrosis Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis
 [81] Fibrosis Normal   Fibrosis Fibrosis Fibrosis Fibrosis Normal   Fibrosis
 [89] Fibrosis Fibrosis Fibrosis Normal   Normal   Fibrosis Fibrosis Fibrosis
 [97] Fibrosis Fibrosis Fibrosis Fibrosis Normal   Normal   Fibr

In [54]:
# Create a design matrix
design <- model.matrix(~0 + labels)
colnames(design) <- levels(labels)

In [55]:
fit <- lmFit(normal_counts, design)

In [56]:
contrast.matrix <- makeContrasts(
    Fibrosis_vs_Normal = `Fibrosis` - Normal,
    levels = design
)

# Apply contrasts to the fit
fit2 <- contrasts.fit(fit, contrast.matrix)

# Empirical Bayes moderation to get p-values
fit2 <- eBayes(fit2)

In [57]:
# Get the top DEGs for the Fibrosis vs Normal comparison
top_genes_fib_vs_norm <- topTable(fit2, coef = "Fibrosis_vs_Normal", adjust.method = "BH", number = Inf)

# View the top DEGs
head(top_genes_fib_vs_norm)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6969,0.4767457,4.207436,11.50608,9.952564e-22,1.731348e-17,38.77566
17075,0.5206691,7.265926,11.08760,1.140214e-20,7.320483e-17,36.39218
4419,0.8110163,7.074491,11.04250,1.482760e-20,7.320483e-17,36.13539
10970,0.5251446,5.361546,11.01330,1.757697e-20,7.320483e-17,35.96911
7725,0.6071783,4.957512,10.97299,2.222858e-20,7.320483e-17,35.73958
1776,0.7526904,4.613934,10.92701,2.905256e-20,7.320483e-17,35.47785


In [58]:
write.csv(top_genes_fib_vs_norm, "DEGs_Fibrosis_from_Normal.csv")

In [59]:
filtered_genes_fib_vs_norm <- top_genes_fib_vs_norm[1:500,]

# View filtered DEGs
head(filtered_genes_fib_vs_norm)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6969,0.4767457,4.207436,11.50608,9.952564e-22,1.731348e-17,38.77566
17075,0.5206691,7.265926,11.08760,1.140214e-20,7.320483e-17,36.39218
4419,0.8110163,7.074491,11.04250,1.482760e-20,7.320483e-17,36.13539
10970,0.5251446,5.361546,11.01330,1.757697e-20,7.320483e-17,35.96911
7725,0.6071783,4.957512,10.97299,2.222858e-20,7.320483e-17,35.73958
1776,0.7526904,4.613934,10.92701,2.905256e-20,7.320483e-17,35.47785


In [60]:
genes_fib_vs_norm_names <- rownames(filtered_genes_fib_vs_norm)

In [61]:
common_genes <- intersect(rownames(normal_counts), genes_fib_vs_norm_names)
selected_normal_counts <- normal_counts[common_genes, ]
head(selected_normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,4.5895546,5.4821690,5.013154,5.0660709,4.6558168,4.299078,4.7529575,5.4095139,4.9937768,4.5285302,⋯,4.85356510,4.5671727,4.98515819,5.2347762,4.9007545,4.923087,5.0044999,5.058062,5.0024535,4.221450
17,5.4498461,5.6265590,5.668470,5.8249308,5.6457475,5.172477,5.5504949,5.4456837,5.8252990,5.6696779,⋯,5.57406609,5.2507565,6.00898948,6.0142514,5.9716329,5.707487,5.7846679,5.655138,5.9198271,5.086468
39,7.9621220,8.1135463,8.351269,8.1821062,8.3109562,7.149491,7.9505994,8.3611480,8.2160940,8.0972352,⋯,8.25672321,7.8754781,8.03891652,8.0412155,8.1943746,8.353489,8.4129625,8.349751,8.4014724,7.200374
58,0.8144082,-0.6402276,0.199094,0.5012863,0.5276482,2.050562,0.8282831,-0.8902029,-0.3387205,0.7454972,⋯,0.03594184,-0.4620669,0.01960947,-0.4101482,-0.3964369,-0.626154,0.6014461,1.338027,0.4160938,2.125326
67,5.5138387,6.0274753,6.073207,5.8621856,5.7559469,4.997887,5.7069400,5.7806501,6.0198660,5.8642174,⋯,5.75658513,5.4751801,5.98185629,5.8731974,5.8628355,5.832565,5.7705851,5.658351,6.0170439,5.071807
121,6.2815691,6.4648079,6.162367,6.2472406,6.1707244,5.744103,6.3216265,6.3755424,6.5432032,6.2974713,⋯,6.50300390,6.2821870,6.55750137,6.5650098,6.1550791,6.374336,6.2230671,6.531799,6.2607083,5.943191


In [62]:
dim(selected_normal_counts)

[1] 500 134

In [63]:
write.csv(selected_normal_counts, "subset_data1.csv")
write.csv(meta_data, "meta_data1.csv")

## Second Machine ##

In [64]:
meta_data <- read.csv("train_meta_data.csv")

In [65]:
meta_data <- subset(meta_data, Simplified_class != 'Normal')

In [66]:
head(meta_data)

,Simplified_class
,<chr>
2,Advanced_fibrosis
5,Non_advanced_Fibrosis
7,Advanced_fibrosis
8,Non_advanced_Fibrosis
9,Non_advanced_Fibrosis
11,Advanced_fibrosis


In [67]:
dim(meta_data)

[1] 90  1

In [68]:
rownames(meta_data)

[1] "2"   "5"   "7"   "8"   "9"   "11"  "12"  "13"  "14"  "15"  "16"  "21" 
[13] "22"  "23"  "24"  "25"  "26"  "28"  "30"  "31"  "32"  "33"  "36"  "37" 
[25] "38"  "39"  "40"  "41"  "42"  "43"  "44"  "45"  "46"  "48"  "49"  "50" 
[37] "51"  "52"  "54"  "56"  "57"  "58"  "62"  "63"  "64"  "67"  "69"  "70" 
[49] "71"  "72"  "74"  "75"  "77"  "78"  "79"  "80"  "81"  "83"  "84"  "85" 
[61] "86"  "88"  "89"  "90"  "91"  "94"  "95"  "96"  "97"  "98"  "99"  "100"
[73] "103" "104" "107" "108" "109" "111" "112" "115" "118" "119" "120" "122"
[85] "123" "124" "125" "127" "128" "129"

In [69]:
fibrosis_normal_counts <- normal_counts[as.integer(rownames(meta_data))]

In [70]:
head(fibrosis_normal_counts)

,DLDR_0081,DLDR_0130,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0095,DLDR_0097,DLDR_0086,DLDR_0149,DLDR_0044,⋯,DLDR_0166,DLDR_0089,DLDR_0141,DLDR_0111,DLDR_0136,DLDR_0134,DLDR_0175,DLDR_0087,DLDR_0155,DLDR_0092
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6.5462994,6.550016,5.9652146,6.436726,6.3378589,6.432976,6.268976,6.7395306,6.7404826,5.689253,⋯,6.8765094,6.9340665,6.6000053,6.4707867,6.0799502,6.1192447,6.85952138,6.6352203,6.693334,6.26633823
2,0.5821648,-1.222374,-0.2056643,1.225274,0.3287041,-1.235636,-2.378483,0.2980298,-0.8093479,-1.246025,⋯,-0.8545833,0.1054011,0.9989488,0.2178724,-0.3480662,-0.4492898,0.03594184,-0.4658174,1.603149,0.08898988
3,3.7520898,4.534941,3.1093774,4.104779,4.6099902,4.061103,4.097251,4.2049204,4.6702399,4.245828,⋯,4.2014221,3.8919974,4.8759379,4.6599428,4.5581065,4.3102366,4.12340468,4.3778794,4.473400,4.31945643
4,4.6451746,4.370763,4.8092861,4.411547,4.3253607,4.620430,4.191373,4.2132597,4.4935629,4.706380,⋯,4.3444346,4.1351484,4.9302925,3.8586881,4.4179435,4.4967901,4.76204644,4.2298584,4.755317,4.38806704
5,3.8408991,3.512952,3.9527651,3.099743,3.3085263,3.682309,3.643885,3.3367102,3.2119626,3.467163,⋯,3.4332830,2.2645997,3.3930267,3.0252273,3.4912316,3.1615746,3.95559973,3.6972213,4.027257,3.97372734
6,3.2010747,2.517867,2.7857226,3.357725,1.9136666,1.506867,2.145079,2.9741402,1.5927505,3.059160,⋯,2.6897372,2.7063051,1.6442839,2.9811721,2.5076855,2.9158915,2.72896408,2.2057995,1.576431,2.97651515


In [71]:
dim(fibrosis_normal_counts)

[1] 17396    90

In [72]:
labels <- factor(meta_data$Simplified_class)

In [73]:
print(labels)

 [1] Advanced_fibrosis     Non_advanced_Fibrosis Advanced_fibrosis    
 [4] Non_advanced_Fibrosis Non_advanced_Fibrosis Advanced_fibrosis    
 [7] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[10] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[13] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[16] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[19] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[22] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[25] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[28] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[31] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
[34] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
[37] Non_advanced_Fibrosis Advanced_fibrosis     Non_advanced_Fibrosis
[40] Advanced_fibrosis     Advanced_fibrosis     Non_advanced_Fibrosis
[43] N

In [74]:
# Create a design matrix
design <- model.matrix(~0 + labels)
colnames(design) <- levels(labels)

In [75]:
fit <- lmFit(fibrosis_normal_counts, design)

In [76]:
contrast.matrix <- makeContrasts(
    AdvancedFibrosis_vs_Fibrosis = `Advanced_fibrosis` - Non_advanced_Fibrosis,
    levels = design
)

# Apply contrasts to the fit
fit2 <- contrasts.fit(fit, contrast.matrix)

# Empirical Bayes moderation to get p-values
fit2 <- eBayes(fit2)

In [77]:
# Get the top DEGs for the Advanced Fibrosis vs Fibrosis comparison
top_genes_adv_vs_fib <- topTable(fit2, coef = "AdvancedFibrosis_vs_Fibrosis", adjust.method = "BH", number = Inf)

# View the top DEGs
head(top_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9231954,8.604058,2.266513e-13,3.942826e-09,19.90756
4770,0.482585,4.4402232,7.784892,1.119579e-11,4.824356e-08,16.20400
12710,1.181496,-0.2294500,7.774583,1.175495e-11,4.824356e-08,16.15769
673,1.485822,1.1041118,7.739811,1.385410e-11,4.824356e-08,16.00156
3296,1.594761,2.1489644,7.739625,1.386628e-11,4.824356e-08,16.00073
12060,1.675825,-0.0107199,7.562489,3.195875e-11,9.265907e-08,15.20726


In [78]:
write.csv(top_genes_adv_vs_fib, "DEGs_Advanced_from_NonAdvanced.csv")

In [79]:
filtered_genes_adv_vs_fib <- top_genes_adv_vs_fib[1:500,]

# View filtered DEGs
head(filtered_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9231954,8.604058,2.266513e-13,3.942826e-09,19.90756
4770,0.482585,4.4402232,7.784892,1.119579e-11,4.824356e-08,16.20400
12710,1.181496,-0.2294500,7.774583,1.175495e-11,4.824356e-08,16.15769
673,1.485822,1.1041118,7.739811,1.385410e-11,4.824356e-08,16.00156
3296,1.594761,2.1489644,7.739625,1.386628e-11,4.824356e-08,16.00073
12060,1.675825,-0.0107199,7.562489,3.195875e-11,9.265907e-08,15.20726


In [80]:
genes_adv_vs_fib_names <- rownames(filtered_genes_adv_vs_fib)

In [81]:
common_genes <- intersect(rownames(fibrosis_normal_counts), genes_adv_vs_fib_names)
selected_normal_counts <- fibrosis_normal_counts[common_genes, ]
head(selected_normal_counts)

,DLDR_0081,DLDR_0130,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0095,DLDR_0097,DLDR_0086,DLDR_0149,DLDR_0044,⋯,DLDR_0166,DLDR_0089,DLDR_0141,DLDR_0111,DLDR_0136,DLDR_0134,DLDR_0175,DLDR_0087,DLDR_0155,DLDR_0092
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
57,0.5821648,-0.2894877,-1.138550,-4.060128,-2.104255,0.01812021,-0.05655446,-1.903604,-1.7422337,-1.039574,⋯,-1.954119,-0.6315645,-4.2864534,0.4935068,-2.0218380,-2.650924,-2.285986,-0.385647,-3.5800732,0.3172589
113,2.9842633,2.3734773,2.714901,2.882387,1.802635,2.11920634,0.79144245,1.924215,2.0175032,3.306201,⋯,2.204310,2.8850110,1.4949063,2.5398005,1.7406627,1.713649,2.728964,1.544451,1.6031486,1.7541227
173,0.9447349,1.8259895,1.234908,1.366137,1.188526,0.94656695,-0.05655446,2.041254,0.7756146,2.172476,⋯,1.896738,3.1173737,0.2371086,2.1987636,0.2152012,0.453413,1.338505,1.624068,0.9434887,0.8904442
209,7.0012221,7.2057963,6.750193,7.459017,7.379157,7.11479286,7.02026114,7.433126,7.5770278,7.142655,⋯,7.233940,7.3864811,7.6162986,6.8790879,7.2807466,7.098946,7.339114,7.366168,7.6124251,7.3409794
227,7.4204683,7.8082936,7.245547,7.685967,8.043204,7.85080609,7.79269424,8.012574,8.1505160,7.201694,⋯,7.846088,8.0286295,8.2602000,7.6621244,7.7030819,7.764607,7.869254,7.942482,7.9550401,7.9623461
232,7.1692726,6.9989939,7.278919,7.887874,7.723881,7.41458689,7.02026114,7.363651,7.6123162,7.838375,⋯,7.585871,6.6216264,7.8178177,7.0580706,7.7930878,7.454723,7.755217,7.485884,7.3390384,6.9069785


In [82]:
dim(selected_normal_counts)

[1] 500  90

In [83]:
write.csv(selected_normal_counts, "subset_data2.csv")
write.csv(meta_data, "meta_data2.csv")